In [105]:
import tensorflow as tf
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format

import copy
import cv2
import os
import math
import scipy as sp
import PIL

# Tensorflow
import tensorflow as tf
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB6, ResNet50V2
#from keras_tuner.tuners import RandomSearch

# import scikitplot as skplt
# from sklearn.metrics import roc_auc_score
# import matplotlib.pyplot as plt

# # split dataset
# import splitfolders

# ignore warnings
import warnings
warnings.filterwarnings(action='ignore')

In [106]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train.csv')
df

,file_name,label
0,001.PNG,9
1,002.PNG,4
2,003.PNG,1
3,004.PNG,1
4,005.PNG,6
...,...,...
718,719.PNG,9
719,720.PNG,6
720,721.PNG,2
721,722.PNG,8


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723 entries, 0 to 722
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  723 non-null    object
 1   label      723 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.4+ KB


In [108]:
df = df.astype({'label':'string'})

In [109]:
label = df['label'].unique()
label

<StringArray>
['9', '4', '1', '6', '5', '8', '7', '2', '3', '0']
Length: 10, dtype: string

#데이터 이미지의 local adress와 label 값을 list에 저장

In [110]:
import os
from glob import glob

def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    
    # get image path
    img_path_list.extend(glob(os.path.join(data_dir, '*.PNG')))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))
        
    # get label
    label_list.extend(df['label'])
                
    return img_path_list, label_list

def get_test_data(data_dir):
    img_path_list = []
    
    # get image path
    img_path_list.extend(glob(os.path.join(data_dir, '*.PNG')))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))
    #print(img_path_list)
    
    return img_path_list

In [111]:
all_img_path, all_label = get_train_data('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train')
test_img_path = get_test_data('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test')

In [112]:
all_img_path[:5]


['/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/001.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/002.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/003.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/004.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/005.PNG']

In [113]:
all_label[:5]

['9', '4', '1', '1', '6']

In [114]:
test_img_path[:5]

['/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test/001.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test/002.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test/003.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test/004.PNG',
 '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test/005.PNG']

# Load Model

In [115]:
mobilenet_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False,  weights="imagenet",input_shape=(224,224,3,))


In [116]:
# batch_size: 한번에 forward & Backword 하는 샘플의 수
batch_size = 32

# Training 수
epochs = 50

# x_col = 'file_name'
# y_col = label

# Weight 조절 parameter
LearningRate = 1e-3 # 0.001
Decay = 1e-6

img_width = 224
img_height = 224

a = mobilenet_model.layers[-1].output
x = tf.keras.layers.GlobalAveragePooling2D()(a)
predictions = Dense(10, activation='softmax')(x)

# Input ~ Output 연결해주기
DeepLearning = Model(inputs = mobilenet_model.input, outputs=predictions)

# learning parameter를 더하여 최종 model compile
DeepLearning.compile(optimizer=
         SGD(lr=LearningRate, decay=Decay, momentum=0.9, nesterov=True), 
         loss='categorical_crossentropy',
         metrics=['acc'])

In [117]:
train_directory = '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/'
test_directory  = '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/test/'
model_directory = '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/model/'
tensorboard_directory = '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/' + 'tensorboard'



In [118]:
# 사진을 제대로 불러오는지 확인
# import os
# import glob
# import cv2
# from google.colab.patches import cv2_imshow

# images = '/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/train/001.PNG'
# image = cv2.imread(images)
# cv2_imshow(image)

In [119]:
DATAGEN_TRAIN = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last",
    validation_split=0.10) # Train / Validation

# Online-augmentation 비적용 Generator (Test용)
DATAGEN_TEST = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last")

In [120]:
# Generator의 instance 생성 (Train)
TRAIN_GENERATOR = DATAGEN_TRAIN.flow_from_dataframe(
    df,
    train_directory,
    x_col = 'file_name',
    y_col = 'label',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode= "categorical", # softmax
    subset = "training")

VALID_GENERATOR = DATAGEN_TRAIN.flow_from_dataframe(
    df,
    train_directory,
    x_col = 'file_name',
    y_col = 'label',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode="categorical",
    subset = "validation")

# Generator의 instance 생성 (Test)
TEST_GENERATOR = DATAGEN_TEST.flow_from_dataframe(
    df,
    test_directory,
    x_col = 'file_name',
    y_col = 'label',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    shuffle = False,
    class_mode='categorical')


# sigmoid / binary / binary_crossentropy
# softmax / categorical / categorical_crossentropy

Found 651 validated image filenames belonging to 10 classes.
Found 72 validated image filenames belonging to 10 classes.
Found 199 validated image filenames belonging to 10 classes.


In [121]:
CP = ModelCheckpoint(filepath=model_directory+'mobilenet-{epoch:03d}-{val_loss:.4f}-{val_acc:.4f}.hdf5',
            monitor='val_acc', verbose=1, save_best_only=True, mode='max')
TB = TensorBoard(log_dir=tensorboard_directory, write_graph=True, write_images=True)
LR = ReduceLROnPlateau(monitor='val_loss',factor=0.8,patience=3, verbose=1, min_lr=1e-8)

CALLBACK = [CP, TB, LR]

In [ ]:
DeepLearning.fit(
        TRAIN_GENERATOR,
        steps_per_epoch=TRAIN_GENERATOR.n / batch_size,
        epochs=50,
        callbacks=CALLBACK,
        shuffle=True, 
        validation_data=VALID_GENERATOR)

Epoch 1/50
21/20 [==============================] - ETA: 0s - loss: 1.7444 - acc: 0.4424
Epoch 1: val_acc improved from -inf to 0.55556, saving model to /content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/model/mobilenet-001-1.1626-0.5556.hdf5
20/20 [==============================] - 31s 1s/step - loss: 1.7444 - acc: 0.4424 - val_loss: 1.1626 - val_acc: 0.5556 - lr: 0.0010
Epoch 2/50
21/20 [==============================] - ETA: 0s - loss: 0.5096 - acc: 0.9140
Epoch 2: val_acc improved from 0.55556 to 0.81944, saving model to /content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/model/mobilenet-002-0.6091-0.8194.hdf5
20/20 [==============================] - 23s 1s/step - loss: 0.5096 - acc: 0.9140 - val_loss: 0.6091 - val_acc: 0.8194 - lr: 0.0010
Epoch 3/50
21/20 [==============================] - ETA: 0s - loss: 0.2054 - acc: 0.9631
Epoch 3: val_acc improved from 0.81944 to 0.83333, saving model to /content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/model/mobi

In [ ]:
DeepLearning.load_weights('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/model/mobilenet-007-0.2495-0.9306.hdf5')


In [ ]:
TEST_Prediction = DeepLearning.predict_generator(TEST_GENERATOR, verbose=1)


In [ ]:
TEST_Prediction[0:5]

In [ ]:
TEST_Prediction.argmax(axis=1)

In [ ]:
Result = pd.DataFrame(TEST_Prediction)*100
Result

In [ ]:
Result[0:5]


#Submission file

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/sample_submission.csv')
submission 

In [ ]:
submission['label'] = TEST_Prediction.argmax(axis=1)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/Mini_Projects/dataset/submission/submission1.csv', index=False)
